In [4]:
import tensorflow as tf
import pandas as pd
import re
import pymysql
from sklearn.model_selection import train_test_split

In [118]:
sql_to_tf_mappings = {
    "avg": "tf.reduce_mean",
    "pow": "tf.pow",
    "log": "tf.math.log",
    "greatest": "tf.maximum",
    "exp": "tf.exp",
    "1/1+exp": "tf.sigmoid",  
    "sum*":"tf.matmul",
}

def extract_view_name(stmt):
    if isinstance(stmt, str) and "create view" in stmt:
        match = re.search(r'create view (\w+)', stmt)
        if match:
            return match.group(1)
    return ""

def extract_select_expressions(stmt):
    if isinstance(stmt, str) and ("select" in stmt or "create view" in stmt):
        match = re.search(r'select (.+?) from', stmt)
        if match:
            return match.group(1).split(' ')
    return []

def create_tf_expression(stmt):
    operators=['+','-','*','/',')','(',',']
    functions=sql_to_tf_mappings.keys()
    ops=[]
    opds=[]
    exp=''
    for st in stmt:
        if st in functions:
            exp=exp+sql_to_tf_mappings[st]
        elif st in operators:
            exp=exp+st
        elif st=='as':
            break
        else:
            st=st.split('.')
            exp=exp+st[0]
    return exp
          
def end_to_end_translate(sql_statements,features_tablename,target_tablename,iterations,learning_rate,db_host,db_user,db_password,db_name,loss_name):
    view_names=[]
    expressions=[]
    tf_expressions=[]
    for sql in sql_statements:
        view_names.append(extract_view_name(sql))
        exp=extract_select_expressions(sql)
        expressions.append(create_tf_expression(exp))
        tf_expressions.append(view_names[-1]+" = "+expressions[-1])
    tf_commands="\n\t\t".join(tf_expressions)
    import_statements=f"""
import tensorflow as tf
import pandas as pd
import pymysql
from sklearn.model_selection import train_test_split
    """
    data_statements=f"""
connection = pymysql.connect(host = {db_host},
                               port = int(3306),
                               user = {db_user},
                               password = {db_password},
                               db = {db_name})
query1 = "SELECT * from {features_tablename}"
housing_data = pd.read_sql(query1, connection)
query2 = "SELECT * from {target_tablename}"
target_data = pd.read_sql(query2, connection)
X_train, X_test, y_train, y_test = train_test_split(housing_data,target_data, test_size=0.2, random_state=42)
X_train = tf.constant(X_train.values, dtype=tf.float32)
y_train = tf.constant(y_train.values, dtype=tf.float32)
y_train = tf.reshape(y_train, [-1, 1])
weights = tf.Variable(tf.random.normal([X_train.shape[1], 1]), name="weights")
bias = tf.Variable(tf.random.normal([1]), name="bias")
learning_rate = {learning_rate}
optimizer = tf.optimizers.Adam(learning_rate)
    
for epoch in range(2000):
    with tf.GradientTape() as tape:
        {tf_commands}
    gradients = tape.gradient(mse, [weights, bias])
    optimizer.apply_gradients(zip(gradients, [weights, bias]))
       
    if epoch % 100 == 0:
       print(f"Epoch: ",epoch, "Loss:",{loss_name}.numpy())
    """
    return import_statements+data_statements

In [126]:
translation=end_to_end_translate(
    [
        "create view y_pred as select sum* ( X_train.featurevalue , weights.weight ) + bias.bias as prediction from housing_data as X_train, weights,bias;",
        "create view squarederror as select pow ( y_pred.prediction - y_train.target , 2 ) as error from y_pred, target_data as y_train;",
        "create view mse as select avg ( squarederror.error ) from squarederror;"
    ],
    features_tablename="features",  
    target_tablename="targets",      
    iterations=1000,
    learning_rate=0.01,
    db_host="localhost",
    db_user="root",
    db_password="123456",
    db_name="housing_data",
    loss_name="mse"
)
with open("generated_linear.py", "w") as f:
    f.write(translation)

translation=end_to_end_translate(
    [
        "create view logits as select sum* ( X_train.featurevalue , weights.weight ) + bias.bias as error from housing_data as X_train, weights,bias;",
        "create view y_pred as select 1/1+exp ( logits.error ) as prediction from logits;",
        "create view bce as select avg ( y_true * log ( y_pred.prediction + 1e-10 ) + (1 - y_train.target ) * log ( 1 - y_pred.prediction + 1e-10 ) ) from target_data as y_train, y_pred;"
    ],
    features_tablename="features",  
    target_tablename="targets",      
    iterations=1000,
    learning_rate=0.01,
    db_host="localhost",
    db_user="root",
    db_password="123456",
    db_name="housing_data",
    loss_name="bce"
)
with open("generated_logistic.py", "w") as f:
    f.write(translation)

translation=end_to_end_translate(
    [
        "create view y_pred as select sum* ( X_train.featurevalue , weights.weight ) + bias.bias as prediction from housing_data as X_train, weights,bias;",
        "create view y_true as select y_train.target * 2 - 1 from target_data as y_train;",
        "create view hinge as select avg ( greatest ( 0.0 , 1 - y_true.target * y_pred.prediction ) ) from y_true,y_pred;"
    ],
    features_tablename="features",  
    target_tablename="targets",      
    iterations=1000,
    learning_rate=0.01,
    db_host="localhost",
    db_user="root",
    db_password="123456",
    db_name="housing_data",
    loss_name="hinge"
)
with open("generated_svm.py", "w") as f:
    f.write(translation)

In [131]:
import pandas as pd
import pymysql

db_name = "housing_data"
db_host = "localhost"
db_username = "root"
db_password = "123456"


connection = pymysql.connect(host = db_host,
                           port = int(3306),
                           user = "root",
                           password = db_password,
                           db = db_name)
# Load the CSV file into a DataFrame
boston_ds = pd.read_csv("BostonHousing.csv")

boston_ds=boston_ds.drop("medv", axis = 1)
cursor = connection.cursor()

#cursor.execute("create database housing_data;")
#cursor.execute("USE housing_data; CREATE TABLE IF NOT EXISTS housing_data (CRIM FLOAT, ZN FLOAT, INDUS FLOAT, CHAS INT, NOX FLOAT, RM FLOAT, AGE FLOAT, DIS FLOAT, RAD INT, TAX FLOAT, PTRATIO FLOAT, B FLOAT, LSTAT FLOAT);")

# Insert each row of data into the MySQL table
for _, row in boston_ds.iterrows():
    sql = """
    INSERT INTO housing_data (CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, B, LSTAT)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, tuple(row))

# Commit the transaction and close the connection
connection.commit()
cursor.close()
connection.close()

In [141]:
import pandas as pd
import pymysql

db_name = "housing_data"
db_host = "localhost"
db_username = "root"
db_password = "123456"


connection = pymysql.connect(host = db_host,
                           port = int(3306),
                           user = "root",
                           password = db_password,
                           db = db_name)
# Load the CSV file into a DataFrame
boston_ds = pd.read_csv("BostonHousing.csv")

boston_target=boston_ds["medv"]
cursor = connection.cursor()

#cursor.execute("USE housing_data; CREATE TABLE target_data (target FLOAT);")

for value in boston_target:
    sql = "INSERT INTO target_data (target) VALUES (%s)"
    cursor.execute(sql, (value,))

# Commit the transaction and close the connection
connection.commit()
cursor.close()
connection.close()

In [179]:
connection = pymysql.connect(host = db_host,
                           port = int(3306),
                           user = "root",
                           password = db_password,
                           db = db_name)
query1 = "SELECT * from housing_data"
housing_data = pd.read_sql(query1, connection)
query2 = "SELECT * from target_data"
target_data = pd.read_sql(query2, connection)
X_train, X_test, y_train, y_test = train_test_split(housing_data,target_data, test_size=0.2, random_state=42)
X_train = tf.constant(X_train.values, dtype=tf.float32)
y_train = tf.constant(y_train.values, dtype=tf.float32)
y_train = tf.reshape(y_train, [-1, 1])
weights = tf.Variable(tf.random.normal([X_train.shape[1], 1]), name="weights")
bias = tf.Variable(tf.random.normal([1]), name="bias")

learning_rate = 0.01
optimizer = tf.optimizers.Adam(learning_rate)

for epoch in range(2000):
    with tf.GradientTape() as tape:
        y_pred = tf.matmul(X_train, weights) + bias
        squarederror = tf.pow(y_pred-y_train,2)
        mse = tf.reduce_mean(squarederror)
    
    gradients = tape.gradient(mse, [weights, bias])
    optimizer.apply_gradients(zip(gradients, [weights, bias]))
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {mse.numpy()}")

X_test = tf.constant(X_test.values, dtype=tf.float32)
y_test = tf.constant(y_test.values, dtype=tf.float32)
y_test = tf.reshape(y_test, [-1, 1])

y_pred_test=tf.matmul(X_test, weights) + bias
#print(y_pred)
test_squarederror = tf.square(y_pred_test - y_test)
test_mse = tf.reduce_mean(test_squarederror)

# Print the test MSE
print(f"Test MSE: {test_mse.numpy()}")

C:\Users\bbsva\AppData\Local\Temp\ipykernel_18296\3310550168.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  housing_data = pd.read_sql(query1, connection)
C:\Users\bbsva\AppData\Local\Temp\ipykernel_18296\3310550168.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  target_data = pd.read_sql(query2, connection)


Epoch 0, Loss: 53127.40625
Epoch 100, Loss: 2662.483642578125
Epoch 200, Loss: 335.15606689453125
Epoch 300, Loss: 250.52830505371094
Epoch 400, Loss: 195.07977294921875
Epoch 500, Loss: 155.789794921875
Epoch 600, Loss: 127.04792785644531
Epoch 700, Loss: 105.34964752197266
Epoch 800, Loss: 88.73339080810547
Epoch 900, Loss: 76.00957489013672
Epoch 1000, Loss: 66.32998657226562
Epoch 1100, Loss: 59.01860809326172
Epoch 1200, Loss: 53.5196533203125
Epoch 1300, Loss: 49.38142013549805
Epoch 1400, Loss: 46.245914459228516
Epoch 1500, Loss: 43.8369140625
Epoch 1600, Loss: 41.9465446472168
Epoch 1700, Loss: 40.421939849853516
Epoch 1800, Loss: 39.15292739868164
Epoch 1900, Loss: 38.06157302856445
Test MSE: 37.74763107299805


In [2]:
import tensorflow as tf
import pandas as pd
import pymysql
from sklearn.model_selection import train_test_split
    
connection = pymysql.connect(host = localhost,
                               port = int(3306),
                               user = root,
                               password = 123456,
                               db = housing_data)
query1 = "SELECT * from features"
housing_data = pd.read_sql(query1, connection)
query2 = "SELECT * from target"
target_data = pd.read_sql(query2, connection)
X_train, X_test, y_train, y_test = train_test_split(housing_data,target_data, test_size=0.2, random_state=42)
X_train = tf.constant(X_train.values, dtype=tf.float32)
y_train = tf.constant(y_train.values, dtype=tf.float32)
y_train = tf.reshape(y_train, [-1, 1])
weights = tf.Variable(tf.random.normal([X_train.shape[1], 1]), name="weights")
bias = tf.Variable(tf.random.normal([1]), name="bias")
learning_rate = 0.01
optimizer = tf.optimizers.Adam(learning_rate)
    
for epoch in range(2000):
    with tf.GradientTape() as tape:
        y_pred = tf.matmul(X_train,weights)+bias
squarederror = tf.pow(y_pred-y_train,2)
mse = tf.reduce_mean(squarederror)
 = 
    gradients = tape.gradient(mse, [weights, bias])
    optimizer.apply_gradients(zip(gradients, [weights, bias]))
       
    if epoch % 100 == 0:
       print(f"Epoch: ",epoch, "Loss:",mse.numpy())
    

TabError: inconsistent use of tabs and spaces in indentation (<string>, line 27)